
# Information about the system & environment

Install `sh` if it doesn't seem to be available.

In [ ]:
from datetime import datetime
import os
import platform
import psutil
try:
    import sh
except ModuleNotFoundError:
    print('installing sh using pip')
    !pip install sh
    import sh
import sys

Note that `psutil` is not in Python's standard library.

## Operating system

Using the `platform` module, it is easy to obtain information on the platform the Python interpreter is running on.

Information about the machine name:

In [ ]:
platform.node()

The architecture and hardware:

In [ ]:
platform.processor()

In [ ]:
sys.byteorder

In [ ]:
os.cpu_count()

In [ ]:
os.sched_getaffinity(0)

The operating system:

In [ ]:
platform.system()

In [ ]:
platform.release()

In [ ]:
platform.version()

In [ ]:
platform.platform()

## Numerics

The properties of floating point numbers can be obtained easily from the `sys.floatinfo` object.

The largest floating point value that can be represented, the smallest positive non-zero value:

In [ ]:
print(sys.float_info.max, sys.float_info.min)

The number of significant digits of a floating point value:

In [ ]:
sys.float_info.dig

## Processes

Detailed information is available on the processes running on the system.

In [ ]:
for process in psutil.process_iter():
    if 'python' in process.name():
        cpu_times = process.cpu_times()
        thread_str = f'threads: {process.num_threads()}'
        cpu_str = f'user: {cpu_times.user}, sys: {cpu_times.system}'
        print(f'{process.pid}: {process.name()} ({thread_str}, {cpu_str})')

CPU times are cumulative over the process' life time.

In [ ]:
for process in psutil.process_iter():
    if process.cpu_times().user > 0.1:
        print(f'{process.name()}: {process.cpu_times().user}')

It is easy to kill processes, so you might want to be careful.

In [ ]:
sleep = sh.sleep(120, _bg=True)

In [ ]:
for process in psutil.process_iter():
    if 'sleep' in process.name():
        print(process)

In [ ]:
name = 'sleep'
killed_pids = []
for process in psutil.process_iter():
    if name == process.name():
        print(f'killing {name}...')
        killed_pids.append(process.pid)
        process.kill()
print('killed: ', ', '.join(map(str, killed_pids)))

## Users

You can retrieve information on the users on the system as well.

In [ ]:
for user in psutil.users():
    started = datetime.strftime(datetime.fromtimestamp(user.started),
                                '%Y-%m-%d %H:%M:%S')
    print(f'{user.name}: {started}')

## Performance

The `psutil` module makes quite some interesting statistics related to system performance available. This can be useful when writing monitoring tools.

The cumulative times for user, nice, system and so on are readily available.

In [ ]:
psutil.cpu_times()

Memory usage can be queried.

In [ ]:
psutil.virtual_memory()

Disk I/O measures such as the total IOP and read/write sizes in byte are easy to obain.

In [ ]:
psutil.disk_io_counters()

Network I/O can similarly be monitored.

In [ ]:
psutil.net_io_counters()

Disk usage for all partitions can be queried.

In [ ]:
for partition in psutil.disk_partitions():
    mountpoint = partition.mountpoint
    if 'snap' not in mountpoint:
        print(f'{mountpoint}: {psutil.disk_usage(mountpoint).percent:.1f}')